<a href="https://colab.research.google.com/github/RhysWangJunfei/PowerFlow/blob/main/bus30_sample_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from matplotlib import pyplot as plt
import sys
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.stats import truncnorm
from scipy.io import loadmat
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model,Sequential,Model
import math
import time
from tensorflow.keras.layers import Conv1D, MaxPooling1D,Lambda,Concatenate
import pdb; 
import scipy

In [17]:
mat_y = loadmat('/content/drive/MyDrive/Phd/opf_gan/Y_bus30.mat')
mat_load = loadmat('/content/drive/MyDrive/Phd/opf_gan/IEEE30_load.mat')
mat_gen = loadmat('/content/drive/MyDrive/Phd/opf_gan/IEEE30_gen.mat')
Y_bus = mat_y['Ybus'].toarray().astype('complex64')
baseMVA = 100
p_capacity_min = 0
p_capacity_max = 3.35
q_capacity_min = -0.95
q_capacity_max = 4.059
load_factor = 0.8
case30_pload = mat_load['aa'][:,2]/baseMVA
case30_qload = mat_load['aa'][:,3]/baseMVA
print(case30_pload)

[0.    0.217 0.024 0.076 0.    0.    0.228 0.3   0.    0.058 0.    0.112
 0.    0.062 0.082 0.035 0.09  0.032 0.095 0.022 0.175 0.    0.032 0.087
 0.    0.035 0.    0.    0.024 0.106]


In [18]:
q_max = mat_gen['gen_attribute'][:,3]/baseMVA
q_min = mat_gen['gen_attribute'][:,4]/baseMVA
p_max = mat_gen['gen_attribute'][:,8]/baseMVA
p_min = mat_gen['gen_attribute'][:,9]/baseMVA
print(p_max)

[0.8  0.8  0.5  0.55 0.3  0.4 ]


In [19]:
def rdm_load_pq_20per(case_pload,case_qload,sample_number):
    number_of_bus = case_pload.shape[0]
    p_demand = np.zeros([sample_number,number_of_bus])
    q_demand = np.zeros([sample_number,number_of_bus])
    for i in range(0,number_of_bus):
        pi = case_pload[i]
        qi = case_qload[i]
        if case_pload[i]!=0:
          p_mw = np.random.uniform(pi*0.8,pi*1.2,sample_number)
          q_mvar = np.random.uniform(qi*0.8,qi*1.2,sample_number)
          p_demand[:,i] = p_mw
          q_demand[:,i] = q_mvar
    return p_demand,q_demand

def rdm_gen_pv(p_min,p_max,sample_number):
  number_of_gen = p_min.shape[0]
  p_supply = np.zeros([sample_number,number_of_gen])
  for i in range(0,number_of_gen):
    p_gen = np.random.uniform(p_min[i],p_max[i],sample_number)
    p_supply[:,i] = p_gen
  vm_gen = np.random.uniform(0.9,1.1,[sample_number,number_of_gen-1])
  return p_supply,vm_gen

In [20]:
p_demand,q_demand = rdm_load_pq_20per(case30_pload,case30_qload,15000*50)
p_supply,vm_gen = rdm_gen_pv(p_min,p_max,15000*50)

In [21]:
p_supply.shape

(750000, 6)

In [22]:
subsetter = np.where((p_supply.sum(axis=1) > p_demand.sum(axis=1)))
p_supply_ok = p_supply[subsetter]
p_demand_ok = p_demand[subsetter]
q_demand_ok = q_demand[subsetter]
vm_gen_ok = vm_gen[subsetter,:] 

In [23]:
vm_gen_ok = vm_gen_ok.reshape([-1,5])

In [24]:
pd.DataFrame(p_demand_ok).to_csv('p_demand2.csv')
pd.DataFrame(q_demand_ok).to_csv('q_demand2.csv')
pd.DataFrame(p_supply_ok).to_csv('p_supply2.csv')
pd.DataFrame(vm_gen_ok).to_csv('vm_gen2.csv')